## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Description,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,-46.1927,168.8643,overcast clouds,57.00,93,96,5.01
1,1,Searcy,US,35.2506,-91.7363,clear sky,66.99,52,1,13.80
2,2,Cap Malheureux,MU,-19.9842,57.6142,overcast clouds,80.01,75,99,5.99
3,3,Pevek,RU,69.7008,170.3133,broken clouds,-26.01,98,80,2.68
4,4,Wajid,SO,3.8093,43.2471,overcast clouds,81.34,57,97,9.86


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
#Forgot, adjust it to a float for use as a number

min_temp = float(input("What's your ideal minimum temperature for this trip?"))
max_temp = float(input("What's your ideal maximum temperature for this trip?"))

#use 50 and 80 for testing data 
print(min_temp, max_temp)

What's your ideal minimum temperature for this trip?50
What's your ideal maximum temperature for this trip?80
50.0 80.0


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Description,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,-46.1927,168.8643,overcast clouds,57.00,93,96,5.01
1,1,Searcy,US,35.2506,-91.7363,clear sky,66.99,52,1,13.80
5,5,Lebu,CL,-37.6167,-73.6500,clear sky,57.63,68,5,13.15
8,8,Ushuaia,AR,-54.8000,-68.3000,scattered clouds,53.60,54,40,12.66
11,11,Shahjahanpur,IN,27.8833,79.9167,clear sky,79.81,9,0,6.31


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        339
City           339
Country        332
Lat            339
Lng            339
Description    339
Max Temp       339
Humidity       339
Cloudiness     339
Wind Speed     339
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Description,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,-46.1927,168.8643,overcast clouds,57.00,93,96,5.01
1,1,Searcy,US,35.2506,-91.7363,clear sky,66.99,52,1,13.80
5,5,Lebu,CL,-37.6167,-73.6500,clear sky,57.63,68,5,13.15
8,8,Ushuaia,AR,-54.8000,-68.3000,scattered clouds,53.60,54,40,12.66
11,11,Shahjahanpur,IN,27.8833,79.9167,clear sky,79.81,9,0,6.31


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Mataura,NZ,57.00,overcast clouds,-46.1927,168.8643,
1,Searcy,US,66.99,clear sky,35.2506,-91.7363,
5,Lebu,CL,57.63,clear sky,-37.6167,-73.6500,
8,Ushuaia,AR,53.60,scattered clouds,-54.8000,-68.3000,
11,Shahjahanpur,IN,79.81,clear sky,27.8833,79.9167,
12,Mar Del Plata,AR,68.00,broken clouds,-38.0023,-57.5575,
15,Puerto Ayora,EC,79.00,scattered clouds,-0.7393,-90.3518,
16,Bredasdorp,ZA,60.80,overcast clouds,-34.5322,20.0403,
18,Peniche,PT,62.01,broken clouds,39.3558,-9.3811,
19,Mahanoro,MG,75.83,light rain,-19.9000,48.8000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
        
# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add this lat & long to params as a string under "location"
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    #https://maps.googleapis.com/maps/api/directions/outputFormat?parameters DIRECTIONS API
    #doesn't match module. Using what we used there
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    #this will pull each row's location and run a "nearby" search for lodging and return json data for each hotel found
    hotels = requests.get(base_url, params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        #Make this rows "Hotel Name" column equal to the name of the hotel at index 0 of the results
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    #Index error: something is wrong with this index 
    except (IndexError):
        print("No hotel name...skipping city")
        #why is there not "pass" at this point in the module code. I thought I saw that somewhere?
    


No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city
No hotel name...skipping city


In [30]:
hotel_df.head(22)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Mataura,NZ,57.00,overcast clouds,-46.1927,168.8643,Ellie's Villa
1,Searcy,US,66.99,clear sky,35.2506,-91.7363,Holiday Inn Express & Suites Searcy
5,Lebu,CL,57.63,clear sky,-37.6167,-73.6500,Hostal Las Lilas
8,Ushuaia,AR,53.60,scattered clouds,-54.8000,-68.3000,Albatros Hotel
11,Shahjahanpur,IN,79.81,clear sky,27.8833,79.9167,OYO 37782 Hotel Shree Mahakali Palace
12,Mar Del Plata,AR,68.00,broken clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
15,Puerto Ayora,EC,79.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
16,Bredasdorp,ZA,60.80,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
18,Peniche,PT,62.01,broken clouds,39.3558,-9.3811,Hotel Star inn Peniche
19,Mahanoro,MG,75.83,light rain,-19.9000,48.8000,BUNGALOW MAHAMANINA


In [37]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df5 = hotel_df[hotel_df['Hotel Name']!= ""]

clean_hotel_df = clean_hotel_df5
#Dropping NaN doens't work because the cells aren't null, they are empty strings!
clean_hotel_df.head(22)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Mataura,NZ,57.00,overcast clouds,-46.1927,168.8643,Ellie's Villa
1,Searcy,US,66.99,clear sky,35.2506,-91.7363,Holiday Inn Express & Suites Searcy
5,Lebu,CL,57.63,clear sky,-37.6167,-73.6500,Hostal Las Lilas
8,Ushuaia,AR,53.60,scattered clouds,-54.8000,-68.3000,Albatros Hotel
11,Shahjahanpur,IN,79.81,clear sky,27.8833,79.9167,OYO 37782 Hotel Shree Mahakali Palace
12,Mar Del Plata,AR,68.00,broken clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
15,Puerto Ayora,EC,79.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
16,Bredasdorp,ZA,60.80,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
18,Peniche,PT,62.01,broken clouds,39.3558,-9.3811,Hotel Star inn Peniche
19,Mahanoro,MG,75.83,light rain,-19.9000,48.8000,BUNGALOW MAHAMANINA


In [39]:
# CONFIRM that df looks like sample in challenge instructions before saving csv

# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

#what does **row refer to? entire row? ah, thanks Artem for a better article: 
#https://treyhunner.com/2018/10/asterisks-in-python-what-they-are-and-how-to-use-them/
#it's saying all the key/value pairs so can pull the value "city" from the key/column City, etc.

#this didn't help but is interesting: https://ryxcommar.com/2020/01/15/for-the-love-of-god-stop-using-iterrows/

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [43]:
# 11a. Add a marker layer for each city to the map. 
#just the marker layer. The cities will already be sorted into good on temperature so I guess you don't need it.
#or because you'll add driving directions in the next deliverable?

#center has to be a tuple. if you say center, you also have to say zoom
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.2)

#set the marker layer for each location with an info box containing hotel info.
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

#add the marker layer to the figure
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

#Take a screenshot of your map and save it to the Vacation_Search folder as WeatherPy_vacation_map.png.

Figure(layout=FigureLayout(height='420px'))